## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [1]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import seaborn as sns
import os

import pinon as pn
import pinon.names as pn_cols
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

In [2]:
config = pn.Config('master', 'V', 10)

Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data
Dataset "us-companies" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (1 days old).


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-income-banks-quarterly" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-income-insurance-quarterly" on disk (1 days old).


...

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-derived-banks-quarterly" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
Dataset "us-derived-insurance-quarterly" on disk (1 days old).


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!
There is an error in the supplied ticker symbol EEFT in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company uses non stardard quarterly reporting date not currently supported. Company was dropped from the list.
There is an error in the supplied ticker symbol PAGS in Company List of sheet: V of config file: /Users/daved/Dev/pinon/config/master.xlsx. Company was not found and was dropped from the list.


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


In [3]:
c = pn.Comps(config)
c.run()

Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... 

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Dataset "us-derived-shareprices-daily" on disk (1 days old).
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Adding breaking report for ticker V for report date 2023-03-31
Adding breaking report for ticker DFS for report date 2023-03-31
Here
Here
Here
Here


In [18]:
temp = c.m.mu_multiples.loc[config.get_peer_list('V')]
temp.index.get_level_values(0)
s = pd.Series(config.get_peer_weights('V'), index=config.get_peer_list('V'))
temp1 = temp.reset_index(level=0)
temp1['New Col'] = 4
temp1.map(s)

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
dp = pn.DailyPrices()
dp.get_downsampled_share_price_ratios('V')[sf_cols.PE_TTM].tail(70).plot()

In [ ]:
c.m.multiples.loc[('AXP',), pn_cols.TTM_PE_RATIO].plot()

In [ ]:
c.m.multiples.loc[('V', '2009-12-31:2010-06-30')]